In [48]:
# remove spark heavy logging
log4j = sc._jvm.org.apache.log4j
log4j.LogManager.getRootLogger().setLevel(log4j.Level.ERROR)

# HDFS Base path for Yelp Data
base_path = '/shared/yelp/json'

# Yelp file types list
file_type_list = ['businesses', 'checkins', 'reviews', 'tips', 'users']

# Dataframe dictionnary for each file type
dataframe_map = {t: sqlContext.read.json(base_path + '/%s' % t)
                 for t in file_type_list}

# Printing dataframes' schema
for t in file_type_list:
    print('Schema for %s:' % t)
    dataframe_map[t].printSchema()
    print('*' * 80)

NameError: name 'sc' is not defined

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DateType
import datetime
import pandas as pd
format_date = udf(lambda date: datetime.datetime.strptime(date, '%Y-%m-%d'), DateType())

In [ ]:
df_reviews = dataframe_map['reviews']
df_users = dataframe_map['users']
df_tips = dataframe_map['tips']
df_reviews = df_reviews.withColumn('date', format_date(df_reviews.date))
df_tips = df_tips.withColumn('date', format_date(df_tips.date))
db = dataframe_map['businesses']
df_businesses = db.where(db.city == 'Las Vegas').sample(False,0.1)

In [ ]:
from sklearn.cluster import KMeans
import numpy as np
import collections as c
from pyspark.sql.types import IntegerType

apply_len = udf(lambda s: len(s), IntegerType())
df_reviews = df_reviews.withColumnRenamed('user_id', 'user_id_r')
alpha = df_users.join(df_reviews, df_users.user_id == df_reviews.user_id_r, 'inner')
alpha = alpha.withColumnRenamed('review_count', 'review_count_alpha');
beta = alpha.join(df_businesses, alpha.business_id == df_businesses.business_id, 'inner')
temp = beta['average_stars','elite', 'fans', 'friends', 'review_count', 'yelping_since', 'user_id']
temp = temp.withColumn('elite_len', apply_len(temp.elite).alias('elite_len'))
temp = temp.withColumn('friends_len', apply_len(temp.friends).alias('friends_len'))
users = temp['average_stars','fans', 'review_count','elite_len', 'friends_len']
users = users.toPandas()
temp = temp['average_stars','fans', 'review_count','elite_len', 'friends_len', 'user_id']
temp = temp.toPandas()
users_norm = (users - users.mean())/(users.max() - users.min())
users_norm = users_norm.as_matrix()
kmeans = KMeans(n_clusters=3, random_state=None).fit(users_norm)
#dummies = pd.get_dummies(users)
#del dummies['automotive']
#print dummies.columns
kmeans.labels_[0:20]
c.Counter(kmeans.labels_)

In [ ]:
#Choix du nombre de clusters => calcul de silhouette moyenne

from sklearn.metrics import silhouette_score

range_n_clusters = [2, 3, 4, 5, 6]

for n_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=n_clusters, random_state=10).fit(users_norm)
    cluster_labels = kmeans.labels_
    silhouette_avg = silhouette_score(users_norm, cluster_labels, sample_size = 5000)
    print("For n_clusters =", n_clusters, "The average silhouette_score is :", silhouette_avg)

In [ ]:
beta.show(10)

In [ ]:
df_users.collect()[2][8]

In [ ]:
# On récupère le nombre de reviews faite par user sur la ville de Las Vegas
reviews_by_user = df_reviews.join(df_businesses,df_reviews.business_id == df_businesses.business_id, 'inner').groupby(['user_id']).count()
tips_by_user = df_tips.join(df_businesses,df_tips.business_id == df_businesses.business_id, 'inner').groupby(['user_id']).count()
reviews_by_user = reviews_by_user.withColumnRenamed('count', 'count_reviews')
tips_by_user = tips_by_user.withColumnRenamed('count', 'count_tips')
tips_by_user = tips_by_user.withColumnRenamed('user_id', 'user_id_t')
count_tips_reviews = reviews_by_user.join(tips_by_user, reviews_by_user.user_id == tips_by_user.user_id_t, 'inner')
count_tips_reviews = count_tips_reviews['user_id', 'count_reviews', 'count_tips']
count_tips_reviews = count_tips_reviews.withColumn('count_total', count_tips_reviews.count_tips+count_tips_reviews.count_reviews)

In [ ]:
nb_reviews = [];
nb_tips = [];
nb_manif = [];
s1 = count_tips_reviews.collect()[8]
s2 = count_tips_reviews.collect()[43]
s3 = count_tips_reviews.collect()[73]
x = sqlContext.createDataFrame([(s1[0], s1[1], s1[2], s1[3]),(s2[0], s2[1], s2[2], s2[3]), (s3[0], s3[1], s3[2], s3[3])])
x = x.withColumnRenamed('_1', 'user_id')
x = x.withColumnRenamed('_2', 'count_reviews')
x = x.withColumnRenamed('_3', 'count_tips')
x = x.withColumnRenamed('_4', 'count_total')
x = x.orderBy(x.count_total, ascending=False)
# x = count_tips_reviews.filter(count_tips_reviews.count_total > 100).orderBy(count_tips_reviews.count_total, ascending = False)
v = x.count();
for k in xrange(v):
    nb_reviews.append(x.collect()[k][1]);
    nb_tips.append(x.collect()[k][2]);
    nb_manif.append(x.collect()[k][3]);
    
x = x.withColumnRenamed('user_id', 'user_id_ys')
df_reviews = df_reviews.withColumnRenamed('business_id', 'business_id_r')
df_tips = df_tips.withColumnRenamed('business_id', 'business_id_t')
df = x.join(df_reviews, df_reviews.user_id_r ==  x.user_id_ys, 'inner')
df2 = df.join(df_businesses, df_businesses.business_id == df.business_id_r, 'inner').orderBy(df.count_total, ascending=False)
df2 = df2['user_id_ys','date', 'business_id', 'count_total']
dfbis = x.join(df_tips, df_tips.user_id ==  x.user_id_ys, 'inner')
df2bis = dfbis.join(df_businesses, df_businesses.business_id == dfbis.business_id_t, 'inner').orderBy(dfbis.count_total, ascending=False)
df2bis = df2bis['user_id_ys','date', 'business_id', 'count_total']

In [ ]:
# Liste pour stocker , pour chaque yelper star, le nombre de fois qu'il a influencé chacun de ses amis
friends_by_user = [];
for i in xrange(len(nb_manif)):
    user_id_yelperstar = x.collect()[i][0];
    reviews_made_by_ys = df2.where(df2.user_id_ys == user_id_yelperstar);
    tips_made_by_ys = df2bis.where(df2bis.user_id_ys == user_id_yelperstar);
    manif_made_by_ys = reviews_made_by_ys.unionAll(tips_made_by_ys);
    # Liste des amis du ième yelper star
    list_friends_user = df_users.where(df_users.user_id == user_id_yelperstar).collect()[0][4];
    specific_friend = [0]*len(list_friends_user);
	for j in xrange(nb_manif[i]):     
        date_manif = manif_made_by_ys.collect()[j][1];
        date_manif_converted = pd.to_datetime(date_manif);
        delta_month = datetime.timedelta(7);
        date_business_manif = date_manif_converted + delta_month;
        x_reviews = df_reviews.where((df_reviews.business_id_r == manif_made_by_ys.collect()[j][2]) & (date_manif <= df_reviews.date) & (df_reviews.date < date_business_manif.to_datetime()));
        x_tips = df_tips.where((df_tips.business_id_t == manif_made_by_ys.collect()[j][2]) & (date_manif < df_tips.date) & (df_tips.date < date_business_manif.to_datetime()));
        x_reviews = x_reviews['business_id_r', 'date', 'user_id_r'];
        x_tips = x_tips['business_id_t', 'date', 'user_id'];
        x_manif = x_reviews.unionAll(x_tips);
        nombre_manif = x_manif.count();
        for k in xrange(nombre_manif):
            if(x_manif.collect()[k][2] in list_friends_user):
                m = list_friends_user.index(x_manif.collect()[k][2]);
                specific_friend[m] = specific_friend[m] + 1;
                print 'Youpi, une influence !'
    
    friends_by_user.append(specific_friend);
    print 'Le user '+x.collect()[i][0]+' a influence '+str(sum(friends_by_user[i]))+' fois ses amis';


In [ ]:
# le seuil de validation correspond au nombre moyen minimal de yelper influencé par une review/tip que l'on accepte 
# pour pouvoir considérer l'individu comme étant effectivement un yelper star
seuil_validation = 1
for i in xrange(len(friends_by_user)):
    if(sum(friends_by_user[i])/nb_manif[i] < seuil_validation):
        print 'FAILURE : Yelper Stars REJECTED...'
        break;
    elif(i == len(friends_by_user) - 1):
        print 'SUCCESS : Yelper Stars VALIDATED'